In [5]:
import requests as r
from datetime import date, timedelta
from typing import List
import pandas as pd
import numpy as np

In [6]:
# Prompt: What day was the game?
# Return list of games

def games_on_day(d: date):
    m, d, y = d.month, d.day, d.year
    assert m >= 1 and m <= 12 and d >= 1 and d <= 31 and y <= 2023 and y >= 1900
    try:
        rawData = r.get(f'https://data.ncaa.com/casablanca/scoreboard/basketball-men/d3/{y}/{str(m).zfill(2)}/{str(d).zfill(2)}/scoreboard.json').json()
        return rawData['games']
    except:
        raise ValueError("Request failed: Invalid date provided")

def find_game_ids_on_day(team: str, d: date) -> List[str]:
    all_games = games_on_day(d)
    filtered_games = list(
        filter(
            lambda game: team.lower() in game['game']['away']['names']['full'].lower() or team.lower() in game['game']['home']['names']['full'].lower() \
                    or team.lower() in game['game']['away']['names']['short'].lower() or team.lower() in game['game']['home']['names']['short'].lower(), 
            all_games
        )
    )
    return [game['game']['gameID'] for game in filtered_games]
    
find_game_ids_on_day("Caltech", date.today())

[]

In [7]:
import pickle

# need to track rebounds, blocks, steals, and possessions
def get_running_pbp(game_id):
    try:
        pbp = r.get(f'https://data.ncaa.com/casablanca/game/{game_id}/pbp.json').json()
        with open("pbp","wb") as f:
            pickle.dump(pbp, f)
        # combine playStats for all periods
        all_stats = [y for x in [p['playStats'] for p in pbp['periods']] for y in x]
        all_stats_df = all_stats
        return all_stats_df
    except:
        raise ValueError("Request failed: Invalid game id")

def set_poss_end(time, home, vis):
    homeOut, visOut = np.zeros((len(home), 1), dtype=np.int32), np.zeros((len(home), 1), dtype=np.int32)
    # this technique misses the first possession
    for i in range(len(home)):
        # set vars for which team is focus of this event
        desc = home[i].lower()
        curr = home
        currOut = homeOut
        otherOut = visOut
        if home[i] == "":
            desc = vis[i].lower()
            curr = vis
            currOut = visOut
            otherOut = homeOut

        # two types of things. either possession ends so for other team possession started
        # spaces around "good" bc what if someone's name is good or smths
        if " good " in desc and "free throw" not in desc:
            otherOut[i+1] = 1 # made a free throw we litty
        elif "defensive rebound " in desc:
            currOut[i] = 1 # this team got possession in that event
        elif "turnover" in desc:
            otherOut[i] = 1 # other team got possession in that event
        elif "free throw good " in desc: 
            # check events after of same timestamp that they are not free throw
            j = i+1
            last = True
            while j < len(time) and time[i] == time[j]:
                if "free throw good " in curr[j].lower():
                    last = False
                    j = len(time)
                j += 1
            if last:
                inc = 1
    return homeOut, visOut

df = get_running_pbp(6060646)
df = pd.DataFrame(df)
df['homeRebounds'] = df['homeText'].str.contains(' REBOUND ', case=False).cumsum()
df['visitorRebounds'] = df['visitorText'].str.contains(' REBOUND ', case=False).cumsum()
df['homeBlocks'] = df['homeText'].str.contains(' Block ', case=False).cumsum()
df['visitorBlocks'] = df['visitorText'].str.contains(' Block ', case=False).cumsum()
df['homeSteals'] = df['homeText'].str.contains(' Steal ', case=False).cumsum()
df['visitorSteals'] = df['visitorText'].str.contains(' Steal ', case=False).cumsum()
df['homePossessions'], df['visitorPossessions'] = set_poss_end(df['time'].to_numpy(), df['homeText'].to_numpy(), df['visitorText'].to_numpy())
df['homePossessions'] = df['homePossessions'].cumsum()
df['visitorPossessions'] = df['visitorPossessions'].cumsum()

pd.set_option("display.max_rows", 100, "display.max_columns", None)
df

,score,time,visitorText,homeText,homeRebounds,visitorRebounds,homeBlocks,visitorBlocks,homeSteals,visitorSteals,homePossessions,visitorPossessions
0,,19:47,"Jumper MISSED by UR's Armstead, Khyree",,0,0,0,0,0,0,0,0
1,,19:45,,"ETBU Defensive REBOUND by Charles, Kevin",1,0,0,0,0,0,1,0
2,0-3,19:35,,"3 Pointer GOOD by ETBU's Williams, Jayden",1,0,0,0,0,0,1,0
3,,19:35,,"ETBU Assist by Elzy, Ryan",1,0,0,0,0,0,1,1
4,,19:10,"Jumper MISSED by UR's Johnson, Nathaniel",,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
692,,00:09,"Subbing out for UR - Brown, Cameron",,38,43,5,0,13,17,79,75
693,,00:09,"Subbing in for UR - Himel, Josh",,38,43,5,0,13,17,79,75
694,91-97,00:09,,"Free Throw GOOD by ETBU's Moore, Darry",38,43,5,0,13,17,79,75
695,,00:05,"UR Turnover by Jackson, Khair",,38,43,5,0,13,17,80,75
